# Lab 2

Note: our lab 2 is the lecture lab 3

Tha aims of the lab are to:
- model the probability of generating language with unigrams and trigram LMs
- evaluate the quality of the language model using perplexity 
- understand and address the issues of sparsity in language modeling
- experiment with applications of language models in understanding text
- 

bigram model -> one character predicts the next one with a lookup table of counts

## Background: Load Linkedin
run the cells below



In [ ]:
import time
import torch
import pandas as pd
import numpy as np
from pathlib import Path


In [ ]:
def load_dataset(file_path) -> pd.DataFrame:
  return pd.read_csv(file_path)

def get_text_as_string(content:pd.Series):
  return content.str.cat(sep=" ")

blokedin_dataset_path = Path(".").resolve().parent / "datasets" / "messages.csv"
print(blokedin_dataset_path, type(blokedin_dataset_path))
blokedin_dataset = load_dataset(blokedin_dataset_path)
blokedin_dataset

Before we get the text as a single string, we need to add the special token `<S>` at the beginning and end of each message so that the n-gram model is able to generate messages that replicate the dataset corpus.

To do this, we need to use a .map function possibly on the df.content Series.

We're not putting the `<s>` at the end. What is the effect of that in our n-gram language model?




In [ ]:
#blokedin_dataset["content"] = "<s>" + blokedin_dataset["content"] #+ "<s>"
# alternative
# blokedin_dataset["content"] = blokedin_dataset["content"].map(lambda message: f"<s> {message}<s>") 

In [ ]:
print(blokedin_dataset["content"].iloc[0])
print(blokedin_dataset["content"].iloc[1])

In [ ]:
# we need this to generate the character's vocab from the content already in our corpus
content = get_text_as_string(blokedin_dataset["content"])
print(len(content))

print(content)


In [ ]:
print(len(set(content)))
content_chars = set(content)


Let's keep the `\\n` in the original text so that our n-gram model can detect that in its language model.

In [ ]:
import string

In [ ]:
print(len(string.printable))

In [ ]:
printable_ch_not_in_content = [ch for ch in string.printable if ch not in content_chars]
printable_ch_not_in_content

In [ ]:
printable_characters_in_content = [ch for ch in string.printable if ch in content_chars]
printable_characters_in_content
"-" in printable_characters_in_content
print(len(printable_characters_in_content), len(string.printable), len(printable_ch_not_in_content))
content_chars_not_in_printable = [ch for ch in content_chars if ch not in string.printable]
print(content_chars_not_in_printable)
print(len(content_chars_not_in_printable))
print("-" in string.printable)
print("-" in content_chars_not_in_printable)
print("-" == content_chars_not_in_printable[11], content_chars_not_in_printable[11], ord(content_chars_not_in_printable[11]), ord("-"))

In [ ]:
special_token = "<s>"
#chars = [special_token] # we'd like to keep this at the beginning
base_vocab = set(string.printable) # i.e. ascii 32 -> 127
assert len(base_vocab) == len(string.printable), f"the base vocab size is not the same as the printable characters count"

chars = sorted(set(content).union(base_vocab)) # returns a list

printable_chars_not_in_chars = [ch for ch in string.printable if ch not in chars]


assert all(char in chars for char in string.printable) == True, f"not all the printable characters are in the chars list {printable_chars_not_in_chars}"

stoi = {char:i+1 for i, char in enumerate(chars)} # I think we should be sorting the vocabulary and add the starting char
stoi[special_token] = 0
itos = {i:char for char,i in stoi.items()} # this is equivalent to the above
# vocabulary = set(content)

# print(len(vocabulary))
# print(vocabulary)


# print(len(vocabulary))
# vocabulary = sorted(vocabulary) # sorted returns a list

print(special_token in chars)

In [ ]:
# just kept the old version to see that you could use another approach too
#vocabulary = {char:i for i, char in enumerate(vocabulary)}

# we need to make sure that all the alphabet chars and numbers are in the vocabulary
# atoi = {chr(i):i for i in range(32, 126)}
# for char, i in atoi.items():
#   if char not in vocabulary:
#     print(f'char={char} not in vocabulary')
#     vocabulary.append(char)

# vocabulary.append(special_token)
# print(vocabulary)

#itos = {i:char for i, char in enumerate(vocabulary)} # same as above

In [ ]:
print(stoi)
print(itos)
print(len(stoi))
print(stoi[special_token])

Question: why did we put the `<s>` in the end? Could have we put it at the start? what would have changed? Also, why do we only have one special character and not one for the end of text?

So the vocab size is 155 with the special_token `<s>` added

In [ ]:
content = blokedin_dataset["content"]
content

In [ ]:
# Note: I think here we can use the Counter() provided by python
from collections import defaultdict
b = defaultdict(int) # named the same as https://github.com/karpathy/nn-zero-to-hero/blob/master/lectures/makemore/makemore_part1_bigrams.ipynb

for message in content:
  characters = [special_token] + list(message) + [special_token]
  for ch1, ch2 in zip(characters, characters[1:]):
    #print(bigram)
    bigram = (ch1, ch2)
    #print(bigram)
    b[bigram] = b.get(bigram, 0) + 1
    #b[bigram] += 1 # I prefer this


# splittt = content.str.split()
# print(splittt)
# type(splittt)

Vocab_counter = dict(sorted(b.items(), key=lambda items: items[1], reverse=True))
print(Vocab_counter)




# this counts the frequencies of each character occuring in our corpus
# create the bigram frequencies
# for message in content.:
#   Vocab_counter[char] += 1

# Vocab_counter = dict(sorted(Vocab_counter.items(), key=lambda items: items[1], reverse=True))
# Vocab_counter

Now we have a bigram counter of frequencies and we need the model to generate the text based on these frequencies. Like create probabilities.

In [ ]:
# there are two approaches here right, create a matrix and or
# in the original karpathy used a 27x27 matrix, we have a bigger vocab size
n = len(chars) + 1
#N = torch.zeros((n, n), dtype=torch.int32) # why did we choose int32?? could have we chosen something unsigned?
N = torch.zeros((n, n), dtype=torch.int32) # why did we choose int32?? could have we chosen something unsigned?

# now for each bigram occurence in the lookup table b we need to calculate the probability
for message in content:
  tokens = [special_token] + list(message) + [special_token]
  for ch1, ch2 in zip(tokens, tokens[1:]):
    idx_ch1 = stoi[ch1]
    idx_ch2 = stoi[ch2]
    #print(idx_ch1, idx_ch2)

    N[idx_ch1, idx_ch2] += 1
    #print(N[idx_ch1, idx_ch2])

N[0, 46]


Let's display this N frequency matrix

In [ ]:
N.size()

In [ ]:
ord(itos[6])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,16))
plt.imshow(N, cmap="Blues")



We can immediately notice that the matrix is very sparse and the majority of occurrences are at the center from more or less index 58 to index 100 where if you have a good eye you can relate that range to the ascii prinatble charactes in the alphabet. This should'n be a suprise as we're dealing with text messages written in english.

We can also see a lot of occurrences in the first few rows, specifically row 6 whic if you look at the itos you notice it's a space and this shouldn't be a surprise either as the space in our messages is used a lot. Now, how do we deal with the space when we sample with this model? will we have very short words? Let's see.




Let's look at the values at row 6, the one with the space followed by a character.

In [ ]:
N[6]

Now let's calculate the probabilities of that row.

In [ ]:
p = N[6].float()
#print(p)
p = p / p.sum()
print(p)

Let's visualize the probabilities for the space followed by a character. What distribution do we see?

In [ ]:
plt.figure(figsize=(16,3))
plt.bar(range(len(p)), p)
plt.show()

Now let's sample from the probabilites using multinomial.

Question, how does the multinomial work?

In [ ]:
# let's create a generator for reproducibility
g = torch.Generator().manual_seed(10000)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
print(ix)
print(itos[ix])

In [ ]:
p = torch.rand(3, generator=g)
print(p)
p = p / p.sum()
p

In [ ]:
genmultinomial = torch.multinomial(p, num_samples=100, replacement=True, generator=g)
genmultinomial

In [ ]:
plt.bar(range(len(genmultinomial)), genmultinomial)
plt.show()
# this should represent the same as the p from torch.random(3)

In [ ]:
p.shape

In [ ]:
P = torch.zeros((n, n), dtype=torch.float16)

In [ ]:
P.shape

In [ ]:
ptrue = P.sum(1, keepdim=True)
print(ptrue)
print(ptrue.shape)


In [ ]:
# 160, 160
# 160,   1

In [ ]:
P.sum(1).shape

In [ ]:
# 160, 160
#   1, 160

In [ ]:
P = (N+1).float()
P /= P.sum(1, keepdims=True)
P

In [ ]:
itos[0]

In [ ]:
for i in range(5):
  out = []
  ix = 0
  while True:
    p = P[ix]
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0: break
  print(''.join(out)) 

Now, this model is really bad, how do we improve it?

- maybe just use printable ascii characters and remove the emojiis occurrences. 
- what is the probability of these before we model it
- would a trigram perform better??
- what is the error (entropy??) of the text before we start modeling this?


Let's implement 1.
